### Import packages

In [1]:
import pandas as pd
from datasets import Dataset

### Import dataset from CSV file

In [2]:
data = pd.read_csv('../backend/data/paris-2024-faq.csv',delimiter=";")

### Extract french language from FAQ

In [3]:
# extract french FAQ
data = data[data['lang']=='fr']

## 1. Recherche sémantique ##

### Combine a new text field for semantic search

In [4]:
data['text'] = data['label'] + ' ' + data['body'] + ' ' + data['topics']

### Import sentence transformers from hugging face

In [5]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

/home/farid/.pyenv/versions/3.10.6/envs/dev-go/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### Save transfomer pickle

In [6]:
import pickle

pickle.dump(tokenizer, open("../backend/data/sentence-transformers_tokenizer.pkl", 'wb'))
pickle.dump(model, open("../backend/data/sentence-transformers_model.pkl", 'wb'))

### Embeddings functions

In [7]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
    encoded_input = tokenizer(text_list, padding=True, truncation=True, return_tensors="pt")
    encoded_input = {k: v for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

### Get embeddings on the text column

In [8]:
text_dataset = Dataset.from_pandas(data)

embeddings_dataset = text_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().numpy()[0]}
)

Map:   0%|          | 0/543 [00:00<?, ? examples/s]

In [41]:
filtered_results = data[data['topics'].str.contains("Recrutement")]
filtered_results = filtered_results.drop(columns={"id","lang","text"})
final_results={}
for k,r in filtered_results.items():
    final_results[k]=r.to_list()

In [42]:
final_results

{'label': ["Je cherche un poste d'agent de sécurité. Il n'y en a pas à pourvoir sur votre site. Où puis-je postuler ?",
  "J'ai un entretien avec un recruteur, j'ai un contretemps. Comment puis-je l'informer et/ou modifier mon rendez-vous ?",
  "Je n'ai pas été retenu. Puis-je postuler à une autre offre ?",
  'Est-il possible de travailler pour Paris 2024 sans maîtriser le français ?',
  'Comment postuler chez Paris 2024 ?',
  'Puis-je envoyer une candidature directement à une adresse e-mail ou postale ?',
  "J'ai postulé et on me demande de répondre à des questions en vidéo. Est-ce obligatoire ?",
  "Quelle est la politique de l'entreprise en matière de télétravail ?",
  'Les postes à pourvoir sont ils en CDI ?',
  "Je n'ai pas été retenu suite à ma candidature pour un emploi. Est-il possible d'être volontaire ?",
  "Quels sont les documents attendus lors d'une candidature ?",
  "Proposez-vous des jobs d'été ?",
  "Je ne trouve pas d'offre de poste me correspondant sur le site officie

### add a faiss index on embeddings dataset

In [9]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'lang', 'label', 'body', 'topics', 'url', 'text', '__index_level_0__', 'embeddings'],
    num_rows: 543
})

In [255]:
embeddings_dataset.filter(lambda x: "Recrutement" in x["topics"])

Filter:   0%|          | 0/543 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'lang', 'label', 'body', 'topics', 'url', 'text', '__index_level_0__', 'embeddings'],
    num_rows: 17
})

In [231]:

        pandaset[pandaset['topics'].str.contains("Recrutement")]

,id,lang,label,body,topics,url,text,__index_level_0__,embeddings
21,lYZw-BDV-fr,fr,Je cherche un poste d'agent de sécurité. Il n'...,Les agents de sécurité seront embauchés par de...,Autres;Recrutement,https://help.paris2024.org/contents/Je-cherche...,Je cherche un poste d'agent de sécurité. Il n'...,21,"[0.326814204454422, -0.26716384291648865, -0.1..."
22,lYe5M_yu-fr,fr,"J'ai un entretien avec un recruteur, j'ai un c...",Si le recruteur vous a invité directement par ...,Autres;Recrutement,https://help.paris2024.org/contents/J-ai-un-en...,"J'ai un entretien avec un recruteur, j'ai un c...",22,"[0.16654978692531586, 0.0013078578049317002, -..."
68,lYjH-EAB-fr,fr,Je n'ai pas été retenu. Puis-je postuler à une...,Il n'est pas utile de postuler à toutes les of...,Autres;Recrutement,https://help.paris2024.org/contents/Je-n-ai-pa...,Je n'ai pas été retenu. Puis-je postuler à une...,68,"[0.1215088739991188, 0.07961368560791016, -0.1..."
69,lYhpdiqf-fr,fr,Est-il possible de travailler pour Paris 2024 ...,La maîtrise du français est obligatoire pour l...,Autres;Recrutement,https://help.paris2024.org/contents/Est-il-pos...,Est-il possible de travailler pour Paris 2024 ...,69,"[0.018843060359358788, 0.04552905634045601, -0..."
71,OnoZ4KWU-fr,fr,Comment postuler chez Paris 2024 ?,"Pour postuler chez Paris 2024, vous devez vous...",Autres;Recrutement,https://help.paris2024.org/contents/Comment-po...,Comment postuler chez Paris 2024 ? Pour postul...,71,"[0.12723709642887115, 0.03959513083100319, -0...."
72,lYWk1dUP-fr,fr,Puis-je envoyer une candidature directement à ...,Seules les candidatures faites via le site car...,Autres;Recrutement,https://help.paris2024.org/contents/Puis-je-en...,Puis-je envoyer une candidature directement à ...,72,"[0.38620978593826294, -0.12024963647127151, -0..."
75,lYbhM-vv-fr,fr,J'ai postulé et on me demande de répondre à de...,La réponse à des questions en vidéo n'est pas ...,Autres;Recrutement,https://help.paris2024.org/contents/J-ai-postu...,J'ai postulé et on me demande de répondre à de...,75,"[0.02693028561770916, -0.2153499573469162, -0...."
210,lYsutsgr-fr,fr,Quelle est la politique de l'entreprise en mat...,La charte de télétravail Paris 2024 permet deu...,Autres;Recrutement,https://help.paris2024.org/contents/Quelle-est...,Quelle est la politique de l'entreprise en mat...,285,"[0.10767537355422974, -0.3406457006931305, -0...."
283,lYmBswMC-fr,fr,Les postes à pourvoir sont ils en CDI ?,Les postes à pourvoir au sein de Paris 2024 so...,Autres;Recrutement,https://help.paris2024.org/contents/Les-postes...,Les postes à pourvoir sont ils en CDI ? Les po...,429,"[-0.08966363221406937, -0.16282516717910767, -..."
285,lYkx1iD--fr,fr,Je n'ai pas été retenu suite à ma candidature ...,La campagne de recrutement des volontaires est...,Autres;Recrutement,https://help.paris2024.org/contents/Je-n-ai-pa...,Je n'ai pas été retenu suite à ma candidature ...,432,"[0.07198700308799744, -0.19156815111637115, -0..."


### Question embedding

In [39]:
question = "flamme"
question_embedding = get_embeddings(question).detach().numpy()
question_embedding.shape

(1, 768)

### Get nearest examples between questions and text 

In [245]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

In [ ]:
### Display results, look ok

In [246]:
samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)
samples_df['label']

4    Faut-il être de nationalité française pour por...
3    Faut-il une condition physique particulière po...
2    D’où vient la Flamme Olympique ? Que représent...
1    Que se passe-t-il si la Flamme Olympique s’éte...
0                      Que devient la Flamme la nuit ?
Name: label, dtype: object

## 2. Recherche catégorie ##

In [196]:
categories = {}
def split_topics(data):
    for v in data:
        split = v.split(';')
        if len(split)>1:
            if split[0] not in categories:
                categories[split[0]]={}
            categories[split[0]][split[1]]={}
        if len(split)>2:
            if split[2] not in categories[split[0]][split[1]]:
                categories[split[0]][split[1]][split[2]]={}
    return categories
    
    

In [191]:
data[data['topics'].str.contains('Je suis volontaire')]['topics']

90     Volontaires;Je suis volontaire
91     Volontaires;Je suis volontaire
474    Volontaires;Je suis volontaire
671    Volontaires;Je suis volontaire
735    Volontaires;Je suis volontaire
Name: topics, dtype: object

In [197]:
split_topics(data['topics'])

{'Autres': {'Terre de Jeux': {'Autre (animation, communication, financement, CPJ, marque)': {}},
  'Comptes et données personnelles': {},
  'Éducation': {'Ma classe aux Jeux': {}},
  'Recrutement': {},
  'CIO et IPC': {},
  'Engagements environnementaux': {'Questions générales': {}},
  'Partenariats': {},
  'Impact2024-Fonds de dotation': {},
  'La marque Paris 2024': {},
  'Contact Barreau de Paris pendant les Jeux': {},
  "Dispositifs d'alerte Paris 2024": {}},
 'Sports et Athlètes': {'Athlètes': {'Le Village des Athlètes': {}},
  'Sports': {'Calendrier des épreuves et diffusion des Jeux': {}}},
 'Relais de la Flamme ': {'Le parcours du Relais de la Flamme': {'Le parcours de la Flamme Olympique': {}},
  'Les porteurs de la Flamme': {},
  'Les Flammes et la torche': {}},
 'Cérémonies et célébrations': {'Célébrations': {'Olympiade Culturelle': {}},
  'Cérémonies': {'Cérémonies Paralympiques': {}}},
 'Club Paris 2024 / Marathon Pour Tous': {'Marathon Pour Tous': {'Questions générales': 